# References
https://github.com/ultimate010/crnn 
{Combination of Convolutional and Recurrent Neural Network for Sentiment Analysis of Short Texts}
https://keras.io/examples/imdb_bidirectional_lstm/

# Folder Paths

In [1]:
first_time = True

In [2]:
folder_path =  '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/'
lankadeepa_data_path = folder_path + 'corpus/new/preprocess_from_isuru/lankadeepa_tagged_comments.csv'
gossip_lanka_data_path = folder_path + 'corpus/new/preprocess_from_unicode_values/gossip_lanka_tagged_comments.csv'
embedding_size = 400
embedding_type = 'word2vec'
embedding_type1 = 'word2vec'
context = 5
word_embedding_path = folder_path + "word_embedding/"+embedding_type+"/source2_data_from_gosspiLanka_and_lankadeepa/"+str(embedding_size)+"/"+embedding_type1+"_"+str(embedding_size)+"_"+str(context)
word_embedding_keydvectors_path = folder_path + "word_embedding/"+embedding_type+"/source2_data_from_gosspiLanka_and_lankadeepa/"+str(embedding_size)+"/keyed_vectors/keyed.kv"
embedding_matrix_path = folder_path + 'Sentiment Analysis/CNN RNN/embedding_matrix/'+embedding_type+'_lankadeepa_gossiplanka_'+str(embedding_size)+'_'+str(context)

# Imports Statements

In [3]:
if (first_time):
  from google.colab import drive
  drive.mount('/content/drive')

import collections
import pickle
import re
import random
import sys
import os 
import time

import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import FastText
from gensim.models import word2vec

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,precision_recall_fscore_support

from __future__ import print_function

import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import cumsum

import keras
from keras.models import Sequential,Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout, Activation, Flatten, \
    Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, \
    Input, Dense, merge, Add,TimeDistributed, Bidirectional,SpatialDropout1D
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2, l1_l2
from keras.constraints import maxnorm
from keras import callbacks
from keras.utils import generic_utils
from keras.optimizers import Adadelta

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


# Load Data

In [4]:
lankadeepa_data = pd.read_csv(lankadeepa_data_path)[:9059]
gossipLanka_data = pd.read_csv(gossip_lanka_data_path)
gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

In [5]:
lankadeepa_data.columns

Index(['docid', 'comment', 'label'], dtype='object')

In [6]:
lankadeepa_data

,docid,comment,label
0,100204,නියම සිංහල මහත්මයෙක්.,4
1,100204,අන්න මිනිස්සු...,4
2,100204,හොරා හොද මිනිහෙක් නම් අනේ මහත්තයෝ සමාවෙන්න කිය...,3
3,100204,තව ඩිංගෙන් ලබු ගෙඩියත් කොස් ගෙඩි සහ පොල් ගෙඩි ...,2
4,100204,බතල වැලට කොස්ගෙඩි අටට උසාවි ගිහින් රිමන්ඩ් කරන...,2
...,...,...,...
9053,460747,එ්ක තමයි මෙයාලා කෑගහන්නේ මේ රජය අසමත් කිය කියා.,2
9054,460747,මේගොල්ලෝ කිසි දවසක හිරේ යන්නේ නැහැ. එ්ක ස්ථීරය...,2
9055,460747,දූෂණ වංචා සම්බන්ධයෙන් චෝදනා ලැබූ පසුගිය ආණ්‌ඩු...,2
9056,460747,වැරදිකාරයන්ට දඬුවම් කරන්න ඕනෑ,2


In [7]:
all_data = pd.concat([lankadeepa_data,gossipLanka_data])

In [8]:
all_data['label'].value_counts()

2    7665
4    3080
3    2403
5    1911
Name: label, dtype: int64

# preprocess Data

In [9]:
# edit this later 
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

# edit this later 
def text_preprocessing_1(data):
  comments = data['comment']
  labels = data['label']

  comments_splitted = []
  labels_encoded = []

  for label in labels:
    if label == "POSITIVE":
      labels_encoded.append(1)
    else:
      labels_encoded.append(0)

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)
  return comments_splitted,labels_encoded


def text_preprocessing_2(data):
  comments = data['comment']
  labels = data['label']

  comments_splitted = []

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)

  return comments_splitted,labels

In [10]:
comment_texts, comment_labels = text_preprocessing_2(all_data)

# prepare tokenizer

t = Tokenizer()
t.fit_on_texts(comment_texts)
vocab_size = len(t.word_index) + 1
print(vocab_size)

60316


In [11]:
encoded_docs = t.texts_to_sequences(comment_texts)
print(encoded_docs)

[[206, 119, 14241], [773, 22240], [857, 93, 774, 2, 49, 678, 5504, 9, 10683, 1176, 774, 2, 22241, 614, 93, 7253, 1784, 1129], [59, 22242, 3311, 22243, 2368, 1998, 33, 537, 1998, 14242, 22244, 1785], [4098, 14243, 22245, 7254, 1349, 104, 14244, 17, 6258, 2486], [1653, 3088, 14245, 3311, 4099, 1998, 5505, 3089, 8677, 3, 1350, 22246, 22247, 37, 98, 87, 8678], [136, 14246, 351, 220, 6259, 1177, 14247, 351, 220, 664, 14, 3, 14248, 22248, 1, 6260, 14249, 3544, 14, 22249, 5506, 37, 351, 24, 3090, 14], [974, 72], [49, 503, 839, 3, 14250, 383, 1845, 22250, 58, 58, 2253, 9, 4482, 22251], [50, 8679, 3312, 22252, 68, 22253, 760, 314, 615, 22254, 8680, 7255, 6261, 39, 197, 665, 951, 3313, 716, 2487, 9, 890, 22255, 4483, 103, 1264, 22256, 4100, 3797, 1846, 22257, 2369, 22258, 297, 117, 303, 22259, 2254, 803, 383, 428, 666, 109, 63, 311, 268, 717, 27, 14], [2254, 159, 99, 510, 291, 4484, 37, 10684, 32, 22260, 22261, 1654, 14251, 804, 8681, 22262, 14252, 498, 227, 61, 320, 7256, 248, 10685, 2910, 6262

In [12]:
max_length = len(max(encoded_docs, key=len))
padded_docs = pad_sequences(encoded_docs, maxlen=max_length)
print(padded_docs)

comment_labels = np.array(comment_labels)
padded_docs = np.array(padded_docs)

[[    0     0     0 ...   206   119 14241]
 [    0     0     0 ...     0   773 22240]
 [    0     0     0 ...  7253  1784  1129]
 ...
 [    0     0     0 ...  1058  2886 10682]
 [    0     0     0 ...  8155  3389 60313]
 [    0     0     0 ... 60314   379 60315]]


In [13]:

X_train, X_test, y_train, y_test = train_test_split(padded_docs, comment_labels, test_size=0.2, random_state=0)
# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)
# comment_labels = np.array(comment_labels)

# Word Embedding

## Generate Embedding Metrix

In [14]:
def generate_embedding_metrix():
  if (embedding_type == 'fasttext'):
    word_embedding_model = FastText.load(word_embedding_path)
  else:
    word_embedding_model = word2vec.Word2Vec.load(word_embedding_path)
    
  word_vectors = word_embedding_model.wv
  word_vectors.save(word_embedding_keydvectors_path)
  word_vectors = KeyedVectors.load(word_embedding_keydvectors_path, mmap='r')

  embeddings_index = dict()
  for word, vocab_obj in word_vectors.vocab.items():
    embeddings_index[word]=word_vectors[word]

  # create a weight matrix for words in training docs
  embedding_matrix = zeros((vocab_size, embedding_size))
  for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

  pickle.dump(embedding_matrix, open(embedding_matrix_path, 'wb'))
  return embedding_matrix

## Load Embedding Matrix

In [15]:
def load_word_embedding_atrix():
  f = open(embedding_matrix_path, 'rb')
  embedding_matrix= np.array(pickle.load(f))
  return embedding_matrix

# Models

## RNN(LSTM/GRU) model

In [16]:
def build_RNN_model():
    main_input = Input(shape=(maxlen, ), dtype='int32', name='main_input')
    embedding  = Embedding(max_features, embedding_dims,
                  weights=[embedding_matrix], input_length=maxlen,
                  name='embedding' ,trainable=False)(main_input)

    embedding = Dropout(0.50)(embedding)

    x = RNN(rnn_output_size)(embedding)

    x = Dense(hidden_dims, activation='relu', init='he_normal',
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(0.10, name='drop')(x)

    output = Dense(6, init='he_normal',
                   activation='softmax', name='output')(x)

    model = Model(input=main_input, output=output)
    model.compile(loss={'output':'sparse_categorical_crossentropy'},
                optimizer=Adadelta(lr=0.95, epsilon=1e-06),
                metrics=["accuracy"])
    print(model.summary())
    return model

## CNN+RNN(LSTM /GRU) model 

In [17]:
def build_CNN_RNN_model():
    main_input = Input(shape=(maxlen, ), dtype='int32', name='main_input')
    embedding  = Embedding(max_features, embedding_dims,
                  weights=[embedding_matrix], input_length=maxlen,
                  name='embedding' ,trainable=False)(main_input)

    embedding = Dropout(0.50)(embedding)

    conv4 = Convolution1D(nb_filter=nb_filter,
                          filter_length=4,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv4')(embedding)
    maxConv4 = MaxPooling1D(pool_length=2,
                             name='maxConv4')(conv4)

    conv5 = Convolution1D(nb_filter=nb_filter,
                          filter_length=5,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv5')(embedding)
    maxConv5 = MaxPooling1D(pool_length=2,
                            name='maxConv5')(conv5)

    x = keras.layers.Concatenate(axis=1)([maxConv4, maxConv5])

    x = Dropout(0.15)(x)

    x = RNN(rnn_output_size)(x)


    x = Dense(hidden_dims, activation='relu', init='he_normal',
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(0.10, name='drop')(x)

    output = Dense(1, init='he_normal',
                   activation='sigmoid', name='output')(x)

    model = Model(input=main_input, output=output)
    model.compile(loss={'output':'binary_crossentropy'},
                optimizer=Adadelta(lr=0.95, epsilon=1e-06),
                metrics=["accuracy"])
    return model

## CNN+BiLSTM

In [18]:
def build_CNN_BiLSTM():
  # main model
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,300,weights=[embedding_matrix],input_length=maxlen)(input)

  conv4 = Convolution1D(nb_filter=nb_filter,
                          filter_length=4,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv4')(embedding)
  maxConv4 = MaxPooling1D(pool_length=2,
                             name='maxConv4')(conv4)

  conv5 = Convolution1D(nb_filter=nb_filter,
                          filter_length=5,
                          border_mode='valid',
                          activation='relu',
                          subsample_length=1,
                          name='conv5')(embedding)
  maxConv5 = MaxPooling1D(pool_length=2,
                            name='maxConv5')(conv5)


  x = keras.layers.Concatenate(axis=1)([maxConv4, maxConv5])

  x = Dropout(0.15)(x)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=0.8),merge_mode='concat')(x)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  model = Dense(300,activation='relu')(model)
  output = Dense(2,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model


## BiLSTM 

In [19]:
def build_BiLSTM_1_1():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=drop_out_value),merge_mode='concat')(embedding)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  model = Dense(300,activation='relu')(model)
  output = Dense(2,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

# final model
def build_BiLSTM_1_2():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  Bidirectional (LSTM (300,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01)),merge_mode='concat')(embedding)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  # model = Dense(300,activation='relu')(model)
  output = Dense(1,activation='sigmoid')(model)
  model = Model(input,output)
  model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

def build_BiLSTM_1_3():
  input = Input(shape=(maxlen,))
  embedding = Embedding(max_features,embedding_dims,weights=[embedding_matrix],input_length=maxlen)(input)

  model =  LSTM (512,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01))(embedding)
  model = LSTM (256,return_sequences=True,dropout=drop_out_value,kernel_regularizer=l2(0.01))(model)
  model = TimeDistributed(Dense(300,activation='relu'))(model)
  model = Flatten()(model)
  # model = Dense(300,activation='relu')(model)
  output = Dense(1,activation='sigmoid')(model)
  model = Model(input,output)
  model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
  return model

# BiLSTM with dropout regularization
def build_BiLSTM_2_1():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dims, input_length=maxlen,weights=[embedding_matrix]))
  model.add(Bidirectional(LSTM (300,dropout=drop_out_value),merge_mode='concat'))
  # model.add(Dropout(drop_out_value))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

# BiLSTM with l2 regularization
def build_BiLSTM_2_2():
  model = Sequential()
  model.add(Embedding(max_features, embedding_dims, input_length=maxlen,weights=[embedding_matrix]))
  model.add(Bidirectional(LSTM (300,kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)),merge_mode='concat'))
  model.add(Dropout(drop_out_value))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model


# Train and Test

In [20]:
def Train_Test_Model(model,X_train, X_test, y_train, y_test):

  print('Training and Testing...')
  test_accs = []
  first_run = True


  acc=[]
  val_acc=[]
  loss=[]
  val_loss=[]
  best_val_acc = 0
  best_test_acc = 0
  for j in range(nb_epoch):
      a = time.time()
      his = model.fit(X_train, y_train,
                      batch_size=batch_size,
                      validation_data=[X_test, y_test],
                      shuffle=True,
                      epochs=1, verbose=verbosity)
      acc+=his.history['accuracy']
      val_acc+=his.history['val_accuracy']
      loss+=his.history['loss']
      val_loss+=his.history['val_loss']
      # print('Epoch %d/%d\t%s' % (j + 1, nb_epoch, str(his.history)))
      if his.history['val_accuracy'][0] >= best_val_acc:
          score, test_acc = model.evaluate(X_test, y_test,
                                      batch_size=batch_size,
                                      verbose=2)
          best_val_acc = his.history['val_accuracy'][0]
          best_test_acc = test_acc
          print('Got best epoch  best val acc is %f test acc is %f' %
                (best_val_acc, best_test_acc))
          if len(test_accs) > 0:
              print('Current avg test acc:', str(np.mean(test_accs)))
      b = time.time()
      cost = b - a
      left = (nb_epoch - j - 1)
      print('One round cost %ds, %d round %ds %dmin left' % (cost, left,
                                                            cost * left,
                                                            cost * left / 60.0))
      test_accs.append(best_test_acc)
      predictions = model.predict(X_test, batch_size=batch_size, verbose=0)

  print('Avg test acc:', str(np.mean(test_accs)))
  return y_test,predictions

# Cross Validation

In [21]:
def Do_Cross_Validation(model,X,y):

  # Define per-fold score containers
  acc_per_fold = []
  loss_per_fold = []

  kfold = KFold(n_splits=folds, shuffle=True)

  fold_no = 1
  inputs = X
  targets = y
  for train, test in kfold.split(inputs, targets):
    model = model

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    his = model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  shuffle=True,
                  epochs=nb_epoch, 
                  verbose=verbosity,
                  validation_split=validation_split)
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print('------------------------------------------------------------------------')

# Plot Graphs

In [22]:
def Plot_graphs():

  epochs=range(len(acc)) # Get number of epochs

  #------------------------------------------------
  # Plot training and validation accuracy per epoch
  #------------------------------------------------
  plt.plot(epochs, acc, 'r')
  plt.plot(epochs, val_acc, 'b')
  plt.title('Training and validation accuracy')
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend(["Accuracy", "Validation Accuracy"])

  plt.figure()

  #------------------------------------------------
  # Plot training and validation loss per epoch
  #------------------------------------------------
  plt.plot(epochs, loss, 'r')
  plt.plot(epochs, val_loss, 'b')
  plt.title('Training and validation loss')
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend(["Loss", "Validation Loss"])

  plt.figure()


  # Expected Output
  # A chart where the validation loss does not increase sharply!

# Main

In [23]:
# embedding_matrix = generate_embedding_metrix()
embedding_matrix = load_word_embedding_atrix()

In [24]:
batch_size = 32 # 64, 128
nb_filter = 200
filter_length = 4 # test with 2,3,4,5
hidden_dims = nb_filter * 2
nb_epoch = 15
RNN = LSTM 
rnn_output_size = embedding_size 
folds = 3
maxlen = 210 #test with other values
max_features = embedding_matrix.shape[0] #vocab_size
embedding_dims = embedding_size
drop_out_value = 0.5 #0.8 #0.3
verbosity = 1
validation_split = 0.2

In [25]:
model = build_RNN_model()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 210)               0         
_________________________________________________________________
embedding (Embedding)        (None, 210, 400)          24126400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 210, 400)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 400)               1281600   
_________________________________________________________________
mlp (Dense)                  (None, 400)               160400    
_________________________________________________________________
drop (Dropout)               (None, 400)               0         
_________________________________________________________________
output (Dense)               (None, 6)                 2406

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, activation="relu", name="mlp", kernel_initializer="he_normal", kernel_constraint=<keras.con..., bias_constraint=<keras.con...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="softmax", name="output", kernel_initializer="he_normal")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("ma..., outputs=Tensor("ou...)`


In [ ]:
labels, predictions = Train_Test_Model(model,X_train, X_test, y_train, y_test)
# Do_Cross_Validation(model,padded_docs,comment_labels)

Training and Testing...
Train on 12047 samples, validate on 3012 samples
Epoch 1/1
 2912/12047 [======>.......................] - ETA: 7:10 - loss: 1.2601 - accuracy: 0.5120

In [ ]:
# precision, recall, fscore, support = precision_recall_fscore_support(labels, predictions)
# print('precision: {}'.format(precision))
# print('recall: {}'.format(recall))
# print('fscore: {}'.format(fscore))
# print('support: {}'.format(support))

In [ ]:
# predictions[2]
# y_test[2]